In [39]:
from wrangling import *

In [40]:
# prediction
import xgboost as xgb

In [41]:

xg=xgb.XGBClassifier(early_stopping_rounds=20,eval_metric='error')
xg.fit(kag_X_train,kag_y_train,eval_set=[(kag_X_train,kag_y_train),(kag_X_test,kag_y_test)])
xg.score(kag_X_test,kag_y_test)
# res=xg.predict(Xtest)

[0]	validation_0-error:0.21043	validation_1-error:0.22401
[1]	validation_0-error:0.19966	validation_1-error:0.21420
[2]	validation_0-error:0.19546	validation_1-error:0.20518
[3]	validation_0-error:0.19327	validation_1-error:0.20361
[4]	validation_0-error:0.19209	validation_1-error:0.20439
[5]	validation_0-error:0.19226	validation_1-error:0.19890
[6]	validation_0-error:0.18923	validation_1-error:0.19576
[7]	validation_0-error:0.18671	validation_1-error:0.19694
[8]	validation_0-error:0.18604	validation_1-error:0.19694
[9]	validation_0-error:0.18755	validation_1-error:0.19576
[10]	validation_0-error:0.18705	validation_1-error:0.19459
[11]	validation_0-error:0.18167	validation_1-error:0.19733
[12]	validation_0-error:0.18167	validation_1-error:0.19733
[13]	validation_0-error:0.18049	validation_1-error:0.19733
[14]	validation_0-error:0.17813	validation_1-error:0.19772
[15]	validation_0-error:0.17813	validation_1-error:0.19772
[16]	validation_0-error:0.17746	validation_1-error:0.19812
[17]	va

[20]	validation_0-error:0.17359	validation_1-error:0.19812
[21]	validation_0-error:0.17275	validation_1-error:0.19812
[22]	validation_0-error:0.17258	validation_1-error:0.19733
[23]	validation_0-error:0.17174	validation_1-error:0.19772
[24]	validation_0-error:0.16989	validation_1-error:0.19890
[25]	validation_0-error:0.16838	validation_1-error:0.19929
[26]	validation_0-error:0.16838	validation_1-error:0.19929
[27]	validation_0-error:0.16804	validation_1-error:0.19890
[28]	validation_0-error:0.16686	validation_1-error:0.19812
[29]	validation_0-error:0.16535	validation_1-error:0.19733


0.8054138877991369

In [42]:
res=xg.predict(Xtest)

In [43]:
xg.best_ntree_limit

11

In [44]:
params={'reg_lambda':[0],
        'learning_rate':[.1,.3,.7,.5,.9],
        'max_depth':[4,5,7,9,11],
        'n_jobs':[-1],
        'n_estimators':[20,5,60,7,200,100]}

In [45]:
xgrd=xgb.XGBClassifier(early_stopping_rounds=20,eval_metric='error')
cv=(model_selection.GridSearchCV(xgrd,params,cv=3,n_jobs=-1)
    .fit(kag_X_train,kag_y_train,
         eval_set=[(kag_X_train,kag_y_train),(kag_X_test,kag_y_test)],
         verbose=50)
    
    )

cv.best_params_

[0]	validation_0-error:0.23095	validation_1-error:0.23146
[6]	validation_0-error:0.19512	validation_1-error:0.20282


{'learning_rate': 0.5,
 'max_depth': 4,
 'n_estimators': 7,
 'n_jobs': -1,
 'reg_lambda': 0}

In [46]:
cvparams={'learning_rate': 0.5,
 'max_depth': 4,
 'n_estimators': 7,
 'n_jobs': -1,
 'reg_lambda': 0}

nxg=xgb.XGBClassifier(**cvparams,early_stopping_rounds=50)
nxg.fit(kag_X_train,kag_y_train,
         eval_set=[(kag_X_train,kag_y_train),(kag_X_test,kag_y_test)],
         verbose=10)
nxg.score(kag_X_test,kag_y_test)

[0]	validation_0-logloss:0.56083	validation_1-logloss:0.56274
[6]	validation_0-logloss:0.43599	validation_1-logloss:0.44742


0.7971753628874069

In [47]:
nxg.fit(Xtrain,ytrain,eval_set=[(kag_X_train,kag_y_train),(kag_X_test,kag_y_test)])
res=nxg.predict(Xtest)

[0]	validation_0-logloss:0.56117	validation_1-logloss:0.56040


[1]	validation_0-logloss:0.50719	validation_1-logloss:0.50500
[2]	validation_0-logloss:0.48366	validation_1-logloss:0.47951
[3]	validation_0-logloss:0.46599	validation_1-logloss:0.46071
[4]	validation_0-logloss:0.45268	validation_1-logloss:0.44775
[5]	validation_0-logloss:0.44517	validation_1-logloss:0.43993
[6]	validation_0-logloss:0.43734	validation_1-logloss:0.43256


hyperopt

In [48]:
from hyperopt import fmin,tpe,hp,STATUS_OK,Trials
from sklearn.metrics import accuracy_score,roc_auc_score

from typing import Any, Dict, Union

def hyperparameter_tuning(hyperparams: Dict[str,Union[float,int]],
                          kag_X_train:pd.DataFrame,
                          kag_y_train:pd.Series,
                          kag_X_test:pd.DataFrame,
                          kag_y_test:pd.Series,
                          early_stopping_rounds:int=50,
                          metric:callable=accuracy_score)->Dict[str, Any]:
    int_vals=['max_depth','reg_alpha']
    hyperparams={k:(int(val) if k in int_vals else val)
           for k,val in hyperparams.items()}
    hyperparams['early_stopping_rounds']= early_stopping_rounds
    model = xgb.XGBClassifier(**hyperparams)
    evaluation=[(kag_X_train,kag_y_train),(kag_X_test,kag_y_test)]

    model.fit(kag_X_train,kag_y_train,
              eval_set=evaluation,
              verbose=False)
    pred=model.predict(kag_X_test)
    score=metric(kag_y_test,pred)
    return{'loss':-score,'status':STATUS_OK,'model':model}

hyperparams={'max_depth':hp.quniform('max_depth',1,8,1),
         'min_child_weight':hp.loguniform('min_child_weight',-2,3),
         'subsample':hp.uniform('subsample',0.5,1),
         'colsample_bytree':hp.uniform('colsample_bytree',0.5,1),
         'reg_alpha':hp.uniform('reg_alpha',0,10),
         'reg_lambda':hp.uniform('reg_lambda',1,10),
         'gamma':hp.loguniform('gamma',-10,10),
         'learning_rate':hp.loguniform('learning_rate',-7,0),
       #   'random_state':42
         }


In [50]:
trials=Trials()
best=fmin(fn=lambda hyperparams: hyperparameter_tuning(hyperparams,kag_X_train,kag_y_train,kag_X_test,kag_y_test),
          space=hyperparams,
          algo=tpe.suggest,
          max_evals=2_000,
          trials=trials)

best

100%|██████████| 2000/2000 [13:07<00:00,  2.54trial/s, best loss: -0.8109062377402904]


{'colsample_bytree': 0.8037808098513046,
 'gamma': 8.519420372081054,
 'learning_rate': 0.2047887166422218,
 'max_depth': 4.0,
 'min_child_weight': 11.28319186084241,
 'reg_alpha': 1.2941127471018976,
 'reg_lambda': 1.976995326435837,
 'subsample': 0.6797060965412611}

In [51]:
# with noisy data
# best={'colsample_bytree': 0.8037808098513046,
#  'gamma': 8.519420372081054,
#  'learning_rate': 0.2047887166422218,
#  'max_depth': 4.0,
#  'min_child_weight': 11.28319186084241,
#  'reg_alpha': 1.2941127471018976,
#  'reg_lambda': 1.976995326435837,
#  'subsample': 0.6797060965412611}

In [35]:
best={'colsample_bytree': 0.7548336147601576,
 'gamma': 0.07384394880603823,
 'learning_rate': 0.04687842221296814,
 'max_depth': 4.0,
 'min_child_weight': 0.32575523925747074,
 'reg_alpha': 0.45553764910317607,
 'reg_lambda': 6.713863245468691,
 'subsample': 0.8380129565326875}

In [52]:
# hyperopt score
best['max_depth']=4
xghpt=xgb.XGBClassifier(**best,n_estimators=500)
xghpt.fit(Xtrain,ytrain)
res=xghpt.predict(Xtest)

In [18]:
# high score
model1=xgb.XGBClassifier(n_estimators=7,max_depth=8)


model1.fit(Xtrain,ytrain)
res=model1.predict(Xtest)

In [53]:
# submission
result=pd.DataFrame({'PassengerId':raw_test.PassengerId,'Transported':res_encoder.inverse_transform(res)})
result.to_csv('submission2.csv',index=False)